In [117]:
import pandas as pd
import numpy as np
df = pd.read_csv('input/train.csv')
size = df.loc[df['track_id'].idxmax()]['track_id']
playlistsForTrack = np.empty([size], dtype=set)
for i in range(size):
    playlistsForTrack[i] = set(df[df['track_id']==i]['playlist_id'])
bestSimilarTracks = np.empty((size), dtype=object)
for i in range(size):
    bestSimilarTracks[i] = np.empty((size, 2), dtype=object)
    bestSimilarTracks[i][i] = [0,i]
    for j in range(i):
        den = ((len(playlistsForTrack[i]))*(len(playlistsForTrack[j])))**0.5+10
        similarity = len(playlistsForTrack[i] & playlistsForTrack[j])/den
        bestSimilarTracks[i][j] = [-1*similarity, j]
    for j in range(i+1, size):
        den = ((len(playlistsForTrack[i]))*(len(playlistsForTrack[j])))**0.5+10
        similarity = len(playlistsForTrack[i] & playlistsForTrack[j])/den
        bestSimilarTracks[i][j] = [-1*similarity, j]
    bestSimilarTracks[i] = bestSimilarTracks[i][bestSimilarTracks[i][:,0].argpartition(10)][0:10]
    bestSimilarTracks[i] = bestSimilarTracks[i][bestSimilarTracks[i][:,0] !=0]#[:,1]
    bestSimilarTracks[i][:,0] = -1*bestSimilarTracks[i][:,0]
df1 = pd.read_csv('input/target_playlists.csv')
playlists = np.array(df1['playlist_id'])

In [73]:
bestSimilarTracks[10000]

array([[1.0, 9107],
       [1.0, 3639],
       [1.0, 5189],
       [0.5, 18560],
       [0.5, 8988],
       [0.5, 6012],
       [0.5, 19930],
       [0.5, 13079],
       [0.46551724137931033, 20120],
       [0.41379310344827586, 4998]], dtype=object)

In [55]:
cont = -1
for playlist in playlists:
    cont = cont + 1
    tracks = np.array(df[df['playlist_id']==playlist]['track_id'])
    submission[cont][0] = playlist
    tracksSet = set(tracks)
    best = {}
    for track in tracks:
        for similar in bestSimilarTracks[track]:
            if not similar[1] in tracksSet:
                if similar[1] in best:
                    best[similar[1]]=best[similar[1]]-similar[0]
                else:
                    best[similar[1]]=-1*similar[0]
    preSorted = [[v, k] for k,v in best.items()]
    best = np.empty((len(preSorted), 2), dtype=object)
    for i in range(len(preSorted)):
        best[i] = preSorted[i]
    best = np.partition(best, 2, axis=0)[0:10]
    best = np.sort(best, axis=1)[:,1]
    submission[cont][1] = best
df2 = pd.DataFrame(submission, columns=['playlist_id','track_ids'])
df2.to_csv('solution.csv')

[[0.14237390821891627 11112]
 [0.1344609332156069 3057]
 [0.13062394901433358 19134]
 [0.10569045769979289 18402]
 [0.08513994056663515 13727]
 [0.08513994056663515 9930]
 [0.08451871400707595 19985]
 [0.07947220750632637 7035]
 [0.07683453113426447 12768]
 [0.07552142524142422 15261]]


In [120]:
import random
submission = np.empty((len(playlists),2), dtype=object)
cont = -1
for playlist in playlists:
    cont = cont + 1
    tracks = np.array(df[df['playlist_id']==playlist]['track_id'])
    submission[cont][0] = playlist
    tracksSet = set(tracks)
    best = {}
    temp = 1
    for track in tracks:
        if cont < 5000:
            temp -= 1/(len(tracks)+1)
        for similar in bestSimilarTracks[track]:
            if not similar[1] in tracksSet:
                if similar[1] in best:
                    best[similar[1]]=best[similar[1]]-similar[0]*temp
                else:
                    best[similar[1]]=-1*similar[0]*temp
    preSorted = [[v, k] for k,v in best.items()]
    best = np.empty((max(11,len(preSorted)), 2), dtype=object)
    for i in range(len(preSorted)):
        best[i] = preSorted[i]
    if len(preSorted) < 11:
        for i in range(len(preSorted), 11):
            best[i] = [0, random.randint(0, 20000)]
        print(best)
    best = best[best[:,0].argpartition(10)][0:10]
    best = best[best[:,0].argsort()][:,1]
    best = str(best)
    submission[cont][1] = best[1:len(best)-1]
df2 = pd.DataFrame(submission, columns=['playlist_id','track_ids'])
df2.to_csv('solution.csv', index=False)

	


[[-0.37411893198355545 6423]
 [-0.3323901645811422 15752]
 [-0.4498299478888841 11174]
 [-0.055609322679992974 10601]
 [-0.055609322679992974 13404]
 [-0.19619408053055995 17312]
 [0 9390]
 [0 4619]
 [0 13209]
 [0 18136]
 [0 5159]]
[[-2.089853798968116 8297]
 [-2.1258423772599233 19205]
 [-1.9798545862196841 2392]
 [-1.3250609745302007 15543]
 [-0.1604046158903532 12173]
 [0 15301]
 [0 12247]
 [0 6412]
 [0 14373]
 [0 10738]
 [0 16537]]
[[-0.8021714643274991 6330]
 [-0.4231749786552863 7045]
 [-0.5440695544418755 17636]
 [-0.6520297621688179 2815]
 [-0.8393567989785411 3075]
 [-0.25261569346610446 901]
 [-0.5550517789983014 8417]
 [-0.10650887573964501 19565]
 [-0.05964098142789024 13917]
 [-0.10048850358425931 3517]
 [0 8112]]
[[-1.6417877977260737 19814]
 [-0.3828503518483357 10008]
 [-0.18582736905058847 2682]
 [-0.7798841645315275 4366]
 [-0.6413862098924658 8356]
 [-0.13190257576467462 8138]
 [-0.13097604490803091 1944]
 [-0.13023743872750054 18830]
 [-0.12653464602823952 6695]
 [0

[[-4.549295364479288 6717]
 [-4.933402755745593 5801]
 [-5.176418252679447 9738]
 [0 5544]
 [0 9864]
 [0 10228]
 [0 2267]
 [0 14107]
 [0 5496]
 [0 18803]
 [0 1339]]
[[-1.3075296343329863 4601]
 [-1.3194107627239524 14986]
 [-3.356724483412853 11608]
 [-0.8951390496913438 5236]
 [-2.1870468955524944 10649]
 [0 10104]
 [0 11918]
 [0 9495]
 [0 46]
 [0 19906]
 [0 9687]]
[[-2.6882246383939674 7133]
 [-2.6620452128969916 6113]
 [-1.1527426073424585 11088]
 [-0.2776523611990982 18017]
 [-0.2611291039620153 3082]
 [-0.462570841619674 14774]
 [-0.3630216032521172 11478]
 [-0.6762806225325103 5540]
 [-0.8141304746258551 16288]
 [-0.2264866194657621 2638]
 [0 17093]]
[[-1.631945671814529 7929]
 [-3.0618256558923838 14823]
 [-2.1487301302194597 4844]
 [-1.2357458789527271 7991]
 [0 19776]
 [0 4124]
 [0 18254]
 [0 19383]
 [0 8643]
 [0 13152]
 [0 13789]]
[[-4.6477313373451254 6146]
 [-1.8224354900857656 9915]
 [-4.317453114954815 2688]
 [-0.8767497787343073 1946]
 [-0.4876131053025904 16716]
 [0 188

In [110]:
a = [1, 2]
a
str(a)[1:len(str(a))-1]

'1, 2'